# Imports

In [64]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import ta
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
from sklearn.ensemble import RandomForestRegressor
import vectorbt as vbt
from datetime import timedelta

# Functions

In [65]:
def pull_data(ticker: str, start_date: str, end_date: str | None) -> pd.DataFrame:
    """
    Pull historical stock data from Yahoo Finance.
    
    Parameters:
    ticker (str): Stock ticker symbol.
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
    pd.DataFrame: DataFrame containing historical stock data.
    """
    price_data = yf.download(ticker, start=start_date, end=end_date, interval='1d')    
    if price_data is None:
        raise ValueError("No data found for the given ticker and date range.")
    price_data = price_data[['Close','High','Low','Volume']]
    return price_data

In [66]:
def calculate_3day_forward_return(price_data: pd.DataFrame) -> pd.DataFrame: # target variable
    """
    Calculate 3-day forward return and add it as a new column.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing stock data with 'Close' prices.
    
    Returns:
    pd.DataFrame: DataFrame with an additional '3day_forward_return' column.
    """
    price_data['3day_forward_return'] = (price_data['Close'].shift(-3) - price_data['Close']) / price_data['Close'] 
    price_data.dropna(inplace=True)
    return price_data

In [67]:
def calculate_RSI(data: pd.DataFrame, window: int = 14) -> pd.DataFrame:
    """
    Calculate Relative Strength Index (RSI) and add it as a new column.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing stock data with 'Close' prices.
    window (int): Window size for RSI calculation.
    
    Returns:
    pd.DataFrame: DataFrame with an additional 'RSI' column.
    """
    close_series = data["Close"].squeeze()
    data["RSI"] = ta.momentum.RSIIndicator(close=close_series, window=window).rsi()
    data.dropna(inplace=True)
    return data
    

In [68]:
def calculate_SMA(data: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    close_series = data["Close"].squeeze()
    data['sma'] = ta.trend.SMAIndicator(close=close_series, window=window).sma_indicator()
    return data

In [69]:
def calculate_EMA(data: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    close_series = data["Close"].squeeze()
    data['ema_5'] = ta.trend.EMAIndicator(close=close_series, window=5).ema_indicator()
    data['ema_20'] = ta.trend.EMAIndicator(close=close_series, window=20).ema_indicator()
    data['ema_5-20'] = data['ema_5'] - data['ema_20']
    data = data.drop(columns = ['ema_5','ema_20'])
    data.dropna(inplace=True)
    return data

In [70]:
def calculate_MACD(data: pd.DataFrame, window_slow: int = 26, window_fast: int = 12, window_sign: int = 9) -> pd.DataFrame:
    """
    Calculate Moving Average Convergence Divergence (MACD) and add it as a new column.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing stock data with 'Close' prices.
    window_slow (int): Slow EMA window size.
    window_fast (int): Fast EMA window size.
    window_sign (int): Signal line EMA window size.
    
    Returns:
    pd.DataFrame: DataFrame with an additional 'MACD' column.
    """
    close_series = data["Close"].squeeze()
    macd_indicator = ta.trend.MACD(close=close_series, window_slow=window_slow, window_fast=window_fast, window_sign=window_sign)
    data["MACD"] = macd_indicator.macd()
    data["MACD_Signal"] = macd_indicator.macd_signal()
    data["MACD_Diff"] = macd_indicator.macd_diff()
    data = data.drop(columns = ['MACD','MACD_Signal'])
    data.dropna(inplace=True)
    return data

In [71]:
def calculate_ATR(data: pd.DataFrame, window: int = 14) -> pd.DataFrame:
    """
    Calculate Average True Range (ATR) and add it as a new column.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing stock data with 'High', 'Low', and 'Close' prices.
    window (int): Window size for ATR calculation.
    
    Returns:
    pd.DataFrame: DataFrame with an additional 'ATR' column.
    """
    close_series = data["Close"].squeeze()
    high_series = data["High"].squeeze()
    low_series = data["Low"].squeeze()
    data['ATR'] = ta.volatility.AverageTrueRange(high=high_series, low=low_series, close=close_series, window=window).average_true_range()
    data.dropna(inplace=True)
    return data

In [72]:
def calculate_OBV(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate On-Balance Volume (OBV) and add it as a new column.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing stock data with 'Close' and 'Volume'.
    
    Returns:
    pd.DataFrame: DataFrame with an additional 'OBV' column.
    """
    close_series = data["Close"].squeeze()
    volume_series = data["Volume"].squeeze()
    obv_indicator = ta.volume.OnBalanceVolumeIndicator(close=close_series, volume=volume_series)
    data["OBV"] = obv_indicator.on_balance_volume()
    data.dropna(inplace=True)
    return data

In [73]:

def calculate_Bollinger_Bands(data: pd.DataFrame, window: int = 20, window_dev: float = 2.0) -> pd.DataFrame:
    """
    Calculate Bollinger Bands %B and Bandwidth.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing stock data with 'Close' prices.
    window (int): Rolling window size.
    window_dev (float): Number of standard deviations for bands.
    
    Returns:
    pd.DataFrame: DataFrame with 'BB_percent' and 'BB_width' columns.
    """
    close_series = data["Close"].squeeze()
    bb_indicator = ta.volatility.BollingerBands(close=close_series, window=window, window_dev=window_dev)
    data["BB_percent"] = bb_indicator.bollinger_pband()
    data.dropna(inplace=True)
    return data

In [74]:

def calculate_MFI(data: pd.DataFrame, window: int = 14) -> pd.DataFrame:
    """
    Calculate Money Flow Index (MFI) and add it as a new column.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing stock data with 'High', 'Low', 'Close', and 'Volume'.
    window (int): Window size for MFI.
    
    Returns:
    pd.DataFrame: DataFrame with an additional 'MFI' column.
    """
    mfi_indicator = ta.volume.MFIIndicator(
        high=data["High"].squeeze(),
        low=data["Low"].squeeze(),
        close=data["Close"].squeeze(),
        volume=data["Volume"].squeeze(),
        window=window
    )
    data["MFI"] = mfi_indicator.money_flow_index()
    data.dropna(inplace=True)
    return data

In [75]:
def calculate_Chaikin_Money_Flow(data: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    """
    Calculate Chaikin Money Flow (CMF).
    
    Parameters:
    data (pd.DataFrame): DataFrame with 'High', 'Low', 'Close', and 'Volume'.
    window (int): Window size for accumulation/distribution.
    
    Returns:
    pd.DataFrame: DataFrame with 'CMF' column.
    """
    cmf = ta.volume.ChaikinMoneyFlowIndicator(
        high=data["High"].squeeze(),
        low=data["Low"].squeeze(),
        close=data["Close"].squeeze(),
        volume=data["Volume"].squeeze(),
        window=window
    )
    data["CMF"] = cmf.chaikin_money_flow()
    data.dropna(inplace=True)
    return data

In [76]:
def calculate_Stochastic_Oscillator(data: pd.DataFrame, window: int = 14, smooth_window: int = 3) -> pd.DataFrame:
    """
    Calculate Stochastic Oscillator (%K and %D).
    
    Parameters:
    data (pd.DataFrame): DataFrame containing stock data with 'High', 'Low', and 'Close'.
    window (int): Lookback window for highest high and lowest low.
    smooth_window (int): Smoothing window for %D line.
    
    Returns:
    pd.DataFrame: DataFrame with 'Stoch_%K' and 'Stoch_%D' columns.
    """
    stoch = ta.momentum.StochasticOscillator(
        high=data["High"].squeeze(),
        low=data["Low"].squeeze(),
        close=data["Close"].squeeze(),
        window=window,
        smooth_window=smooth_window
    )
    data["Stoch_%D"] = stoch.stoch_signal()
    data.dropna(inplace=True)
    return data

# Creating Dataset

In [120]:
data = pull_data('AMZN', '2024-01-01', None)


C:\Users\lovet\AppData\Local\Temp\ipykernel_6216\1436903007.py:13: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [121]:
data

Price,Close,High,Low,Volume
Ticker,AMZN,AMZN,AMZN,AMZN
Date,,,,
2024-01-02,149.929993,152.380005,148.389999,47339400
2024-01-03,148.470001,151.050003,148.330002,49425500
2024-01-04,144.570007,147.380005,144.050003,56039800
2024-01-05,145.240005,146.589996,144.529999,45153100
2024-01-08,149.100006,149.399994,146.149994,46757100
...,...,...,...,...
2025-11-07,244.410004,244.899994,238.490005,46374300
2025-11-10,248.399994,251.750000,245.589996,36476500


In [122]:
price_data = data.copy()
price_data.columns = data.columns.get_level_values(0)
price_data = price_data.drop(columns=['Price'], errors='ignore')
price_data.head()


Price,Close,High,Low,Volume
Date,,,,
2024-01-02,149.929993,152.380005,148.389999,47339400
2024-01-03,148.470001,151.050003,148.330002,49425500
2024-01-04,144.570007,147.380005,144.050003,56039800
2024-01-05,145.240005,146.589996,144.529999,45153100
2024-01-08,149.100006,149.399994,146.149994,46757100


In [123]:
price_data = calculate_3day_forward_return(price_data) # target variable
price_data = calculate_RSI(price_data)
price_data = calculate_MACD(price_data)
price_data = calculate_EMA(price_data)
price_data = calculate_SMA(price_data)
price_data = calculate_ATR(price_data)
price_data = calculate_OBV(price_data)
price_data = calculate_Bollinger_Bands(price_data)
price_data = calculate_MFI(price_data)
price_data = calculate_Stochastic_Oscillator(price_data)
price_data = calculate_Chaikin_Money_Flow(price_data)
price_data['previous_close'] = price_data['Close'].shift(1)
price_data.dropna(inplace=True)

# Modelling

In [124]:
# --- Define Hyperparameters for the Rolling Window ---

TRAIN_WINDOW_DAYS = 126  # Approx. 6 months of trading days (21 * 6)
# Size of the new data block to test/validate on
TEST_WINDOW_DAYS = 21    # Approx. 1 month of trading days
# The columns to exclude from features
TARGET_COLUMN = '3day_forward_return'

# --- Predictions Storage
all_preds = pd.Series(dtype=float)

# --- The Rolling Window Loop ---
results = []
# Calculate the maximum starting index for the training window
max_train_start_index = len(price_data) - TRAIN_WINDOW_DAYS - TEST_WINDOW_DAYS + 1

# Iterate through the dataset
for i in range(0, max_train_start_index, TEST_WINDOW_DAYS):
    # 1. Define Training Slice
    train_start_idx = i
    train_end_idx = i + TRAIN_WINDOW_DAYS
    train_data = price_data.iloc[train_start_idx:train_end_idx]
    fold_num = 1
    test_dates = price_data.index[train_end_idx:train_end_idx + TEST_WINDOW_DAYS]

    # 2. Define Testing Slice (immediately after training)
    test_start_idx = train_end_idx
    test_end_idx = train_end_idx + TEST_WINDOW_DAYS
    test_data = price_data.iloc[test_start_idx:test_end_idx]

    # --- Prepare X/y splits for this fold ---
    X_train = train_data.drop(columns=[TARGET_COLUMN])
    y_train = train_data[TARGET_COLUMN]
    X_test = test_data.drop(columns=[TARGET_COLUMN])
    y_test = test_data[TARGET_COLUMN]

    # -- Scaling
    scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # --- Model Training and Testing (Placeholder) ---
    # 1. TRAIN your kNN model here using X_train, y_train

    def objective(trial):
        n_neighbors = trial.suggest_int('n_neighbors', 5, 20)
        weights = trial.suggest_categorical('weights', ['uniform','distance'])
        algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
        leaf_size = trial.suggest_int('leaf_size', 10, 50)
        model = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm, leaf_size=leaf_size)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        return r2
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)
    print("Best hyperparameters: ", study.best_params)
    print("Best R^2: ", study.best_value)

    # train set evaluation

    knn = KNeighborsRegressor(**study.best_params)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test) 

    y_pred_series = pd.Series(y_pred, index=test_dates)
    all_preds = pd.concat([all_preds, y_pred_series])
    
    mae_test = mean_absolute_error(y_test, y_pred)
    r2_test = r2_score(y_test, y_pred)

    results.append({
        'fold': len(results),
        'mae_test': mae_test,
        'r2_test': r2_test,
        'best_params': study.best_params,
        'test_dates': data.index[test_start_idx:test_end_idx]
    })


print(f"\nCompleted {len(results)} walk-forward folds.")

Best hyperparameters:  {'n_neighbors': 17, 'weights': 'uniform', 'algorithm': 'kd_tree', 'leaf_size': 41}
Best R^2:  -4.27343860609054


C:\Users\lovet\AppData\Local\Temp\ipykernel_6216\1989930295.py:68: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.



Best hyperparameters:  {'n_neighbors': 19, 'weights': 'uniform', 'algorithm': 'brute', 'leaf_size': 37}
Best R^2:  -0.16701531830234506
Best hyperparameters:  {'n_neighbors': 20, 'weights': 'uniform', 'algorithm': 'brute', 'leaf_size': 29}
Best R^2:  -0.26806646677381174
Best hyperparameters:  {'n_neighbors': 20, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 31}
Best R^2:  -0.17711250793955458
Best hyperparameters:  {'n_neighbors': 15, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 16}
Best R^2:  0.3960515919420654
Best hyperparameters:  {'n_neighbors': 20, 'weights': 'uniform', 'algorithm': 'auto', 'leaf_size': 26}
Best R^2:  -0.0982287189878932
Best hyperparameters:  {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'brute', 'leaf_size': 37}
Best R^2:  0.09904286581260258
Best hyperparameters:  {'n_neighbors': 20, 'weights': 'uniform', 'algorithm': 'auto', 'leaf_size': 39}
Best R^2:  -0.19864131658526807
Best hyperparameters:  {'n_neighbors': 14, 'weigh

In [125]:
for res in results:
    print(res)

{'fold': 0, 'mae_test': 0.039103407279431145, 'r2_test': -4.27343860609054, 'best_params': {'n_neighbors': 17, 'weights': 'uniform', 'algorithm': 'kd_tree', 'leaf_size': 41}, 'test_dates': DatetimeIndex(['2024-07-03', '2024-07-05', '2024-07-08', '2024-07-09',
               '2024-07-10', '2024-07-11', '2024-07-12', '2024-07-15',
               '2024-07-16', '2024-07-17', '2024-07-18', '2024-07-19',
               '2024-07-22', '2024-07-23', '2024-07-24', '2024-07-25',
               '2024-07-26', '2024-07-29', '2024-07-30', '2024-07-31',
               '2024-08-01'],
              dtype='datetime64[ns]', name='Date', freq=None)}
{'fold': 1, 'mae_test': 0.04880390253231927, 'r2_test': -0.16701531830234506, 'best_params': {'n_neighbors': 19, 'weights': 'uniform', 'algorithm': 'brute', 'leaf_size': 37}, 'test_dates': DatetimeIndex(['2024-08-02', '2024-08-05', '2024-08-06', '2024-08-07',
               '2024-08-08', '2024-08-09', '2024-08-12', '2024-08-13',
               '2024-08-14', '20

In [126]:
price_data['predicted_return'] = all_preds
price_data

Price,Close,High,Low,Volume,3day_forward_return,RSI,MACD_Diff,ema_5-20,sma,ATR,OBV,BB_percent,MFI,Stoch_%D,CMF,previous_close,predicted_return
Date,,,,,,,,,,,,,,,,,
2024-08-08,165.800003,166.690002,162.550003,44616200,0.026719,33.589369,-2.330206,-10.958854,180.128001,6.833138,-208700800,0.149797,45.467713,30.494993,-0.010405,162.770004,NaN
2024-08-09,166.940002,168.550003,165.850006,36401000,0.018929,35.311714,-1.867729,-9.838350,178.750501,6.541485,-172299800,0.206355,44.954568,34.778150,-0.002209,165.800003,NaN
2024-08-12,166.800003,168.550003,166.110001,30072800,0.064688,35.191010,-1.446312,-8.883495,177.454501,6.248522,-202372600,0.229282,43.302361,38.223476,-0.003296,166.940002,NaN
2024-08-13,170.229996,171.039993,167.100006,39237900,0.040122,40.552517,-0.841598,-7.208870,176.315001,6.105055,-163134700,0.335964,48.855352,42.010766,0.041121,166.800003,NaN
2024-08-14,170.100006,172.279999,168.860001,28843800,0.047737,40.416057,-0.378969,-6.000876,175.423502,5.913266,-191978500,0.351571,54.047169,44.712316,0.048735,170.229996,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-04,249.320007,257.010010,248.660004,51546300,-0.019694,66.831001,3.713643,12.906143,225.377501,7.459437,731044300,1.032822,67.172635,84.970375,-0.210550,254.000000,-0.022092
2025-11-05,250.199997,251.000000,246.160004,40610700,-0.007194,67.287293,3.708855,13.502955,226.626501,7.272334,771655000,0.972688,66.837593,84.387912,-0.206434,249.320007,-0.020161
2025-11-06,243.039993,250.380005,242.169998,46004200,0.024934,60.048991,3.030779,11.729508,227.391500,7.339310,725650800,0.801586,66.338381,76.087864,-0.273438,250.199997,-0.015385


In [137]:
price_data['predicted_return'].loc['2025-02-10': '2025-03-28']

Date
2025-02-10    0.001957
2025-02-11    0.005716
2025-02-12    0.005545
2025-02-13    0.005545
2025-02-14    0.005545
2025-02-18    0.007479
2025-02-19    0.009416
2025-02-20    0.013707
2025-02-21    0.014602
2025-02-24    0.016562
2025-02-25    0.018296
2025-02-26    0.016860
2025-02-27    0.016860
2025-02-28    0.018595
2025-03-03    0.024317
2025-03-04    0.024317
2025-03-05    0.021949
2025-03-06    0.024739
2025-03-07    0.028088
2025-03-10    0.032542
2025-03-11    0.030993
2025-03-12   -0.002854
2025-03-13   -0.002854
2025-03-14   -0.003193
2025-03-17   -0.004110
2025-03-18   -0.003200
2025-03-19   -0.003029
2025-03-20   -0.001582
2025-03-21   -0.001416
2025-03-24    0.012682
2025-03-25    0.018504
2025-03-26    0.013872
2025-03-27    0.005675
2025-03-28    0.007734
Name: predicted_return, dtype: float64

In [127]:
price_data[['3day_forward_return', 'predicted_return']]

Price,3day_forward_return,predicted_return
Date,,
2024-08-08,0.026719,NaN
2024-08-09,0.018929,NaN
2024-08-12,0.064688,NaN
2024-08-13,0.040122,NaN
2024-08-14,0.047737,NaN
...,...,...
2025-11-04,-0.019694,-0.022092
2025-11-05,-0.007194,-0.020161
2025-11-06,0.024934,-0.015385


In [128]:
price_data['predicted_return'].count()

np.int64(189)

# Signal Generation

In [170]:
def generate_signals(data: pd.DataFrame, results: list) -> pd.DataFrame : 
    """
    Vectorized generation of trading signals based on KNN predictions and fold R2 values.

    Parameters:
        data (pd.DataFrame): Must contain columns:
            - 'Close': current price
            - 'predicted_return': predicted return from KNN
            - 'Date': datetime of price
        results (list): List of dictionaries, each containing 'r2_test'.

    Returns:
        pd.DataFrame: Signals with columns:
            - 'Date': entry date
            - 'Side': 'long' or 'short'
            - 'Entry': entry price
            - 'StopLoss': stop loss price
            - 'ExitDate': exit date
    """
    r2_series = pd.Series(index=data.index, dtype=float)
    

    for i, res in enumerate(results):
        r2_value = res['r2_test']
        test_dates = res['test_dates'] 
        common_idx = r2_series.index.intersection(test_dates)
        r2_series.loc[common_idx] = r2_value

    # Mask for favourable conditions
    #favourable_mask = r2_series > 0.2

    # Masks for long and short entries
    #long_mask = favourable_mask & (data['predicted_return'] >= 0.01)
    #short_mask = favourable_mask & (data['predicted_return'] <= -0.01)

    signals = pd.DataFrame(index=data.index)
    signals['entry'] = data['Close']

    signals['long_entries'] = data['predicted_return'] >= 0.01
    signals['short_entries'] = data['predicted_return'] <= -0.01

    # Build signals DataFrame
    signals['long_exits'] = False
    signals['short_exits'] = False

    # Compute StopLoss
    # Initialize StopLoss column as NaN
    signals['StopLoss'] = float('nan')

    # Long positions: 0.995 * Entry
    signals.loc[signals['long_entries'] == True, 'StopLoss'] = signals['entry'] * 0.995

    # Short positions: 1.05 * Entry
    signals.loc[signals['short_entries'] == True, 'StopLoss'] = signals['entry'] * 1.05


    # Compute ExitDate (3 days after entry)
    for idx in signals.index[signals['long_entries']]:
        exit_idx = signals.index.get_loc(idx) + 3
        if exit_idx < len(signals):
            signals.iloc[exit_idx, signals.columns.get_loc('long_exits')] = True

    for idx in signals.index[signals['short_entries']]:
        exit_idx = signals.index.get_loc(idx) + 3
        if exit_idx < len(signals):
            signals.iloc[exit_idx, signals.columns.get_loc('short_exits')] = True

    # Reset index for clean DataFrame
    return signals.reset_index(drop=True)


In [171]:
signals = generate_signals(price_data, results)

In [172]:
signals

,entry,long_entries,short_entries,long_exits,short_exits,StopLoss
0,165.800003,False,False,False,False,NaN
1,166.940002,False,False,False,False,NaN
2,166.800003,False,False,False,False,NaN
3,170.229996,False,False,False,False,NaN
4,170.100006,False,False,False,False,NaN
...,...,...,...,...,...,...
311,249.320007,False,True,False,False,261.786008
312,250.199997,False,True,False,True,262.709997
313,243.039993,False,True,False,True,255.191993
314,244.410004,False,True,False,True,256.630504


In [142]:
df = signals['entry_signal'].dropna()
df

133     long
134     long
135     long
136     long
137     long
       ...  
310    short
311    short
312    short
313    short
314    short
Name: entry_signal, Length: 82, dtype: object

In [ ]:
pf = vbt.Portfolio.from_signals(
    close=signals['entry'],
    entries=signals['long_entries'],
    exits=signals['long_exits'],
    short_entries=signals['short_entries'],
    short_exits=signals['short_exits'],
    size=1,
    fees=0.001
)

print(pf.stats().to_frame())

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mnon-precise type array(pyobject, 1d, C)[0m
[0m[1mDuring: typing of argument at c:\Users\lovet\AppData\Local\Programs\Python\Python313\Lib\site-packages\vectorbt\portfolio\nb.py (2375)[0m
[1m
File "..\..\..\..\AppData\Local\Programs\Python\Python313\Lib\site-packages\vectorbt\portfolio\nb.py", line 2375:[0m
[1mdef dir_enex_signal_func_nb(c: SignalContext,
    <source elided>

[1m@njit
[0m[1m^[0m[0m

[0m[1mDuring: Pass nopython_type_inference[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function ls_enex_signal_func_nb at 0x0000017FB2A5C360>))[0m
[0m[1mDuring: typing of call at c:\Users\lovet\AppData\Local\Programs\Python\Python313\Lib\site-packages\vectorbt\portfolio\nb.py (2126)[0m
[1m
File "..\..\..\..\AppData\Local\Programs\Python\Python313\Lib\site-packages\vectorbt\portfolio\nb.py", line 2126: